In [172]:
import collections
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from math import log10
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

training_set = pd.read_csv('covid_training.tsv', delimiter='\t')
test_set = pd.read_csv('covid_test_public.tsv', delimiter='\t')


training_set

,tweet_id,text,q1_label,q2_label,q3_label,q4_label,q5_label,q6_label,q7_label
0,1241025578527903750,For the average American the best way to tell ...,no,NaN,NaN,NaN,NaN,no_joke_or_sarcasm,no_not_interesting
1,1240467080954228737,this is fucking bullshit,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
2,1240716889162018816,Can y’all please just follow the government’s ...,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
3,1241062514886090754,No offense but the corona virus disappearing b...,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
4,1241447017945223169,This is the face of someone who just spent 9 h...,yes,2_no_probably_contains_no_false_info,4_yes_probably_of_interest,1_no_definitely_not_harmful,yes_not_urgent,no_not_harmful,no_not_interesting
...,...,...,...,...,...,...,...,...,...
394,1237310167814791168,It's like suicide but minus the haram part I'm...,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting
395,1237460460749766657,The first Silicon Valley death from COVID-19 w...,yes,2_no_probably_contains_no_false_info,4_yes_probably_of_interest,2_no_probably_not_harmful,no_too_trivial_to_check,no_not_harmful,no_not_interesting
396,1237068715314892802,BREAKTHROUGH: Chloroquine phosphate -- an old ...,yes,2_no_probably_contains_no_false_info,5_yes_definitely_of_interest,4_yes_probably_harmful,yes_very_urgent,yes_bad_cure,yes_discusses_cure
397,1237395939171561472,I know everyone is freaking out about COVID-19...,no,NaN,NaN,NaN,NaN,no_not_harmful,no_not_interesting


In [173]:
ids = training_set['tweet_id']#.values
tweets = training_set['text']#.values
training_cv = CountVectorizer()
sparse = training_cv.fit_transform(tweets)
df = pd.DataFrame(sparse.todense(), columns=training_cv.get_feature_names(), index=ids)

df

,00,000,005,07fsra2ny5,0am2j0zhfe,10,100,10k,10mins,11,...,ªgermany,ªsweden,ªð,³ð,¹austria,ºð,¼19,孙春兰,우리의응원을이어주세요,힘내세요_아미
tweet_id,,,,,,,,,,,,,,,,,,,,,
1241025578527903750,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1240467080954228737,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1240716889162018816,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1241062514886090754,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1241447017945223169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237310167814791168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1237460460749766657,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1237068715314892802,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [174]:
test= df.sum(axis=0)
to_remove = test[test <= 1]
df2 = df.drop(columns=to_remove.index)
df2

,00,000,10,100,11,12,14,15,16,17,...,wuhan,year,years,yes,yet,you,young,your,yourself,youâ
tweet_id,,,,,,,,,,,,,,,,,,,,,
1241025578527903750,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1240467080954228737,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1240716889162018816,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1241062514886090754,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1241447017945223169,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1237310167814791168,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1237460460749766657,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,1,0,0,0
1237068715314892802,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [175]:
alpha = 0.01


yes_ids = training_set[training_set['q1_label'] == 'yes']['tweet_id'].values
no_ids = training_set[training_set['q1_label'] == 'no']['tweet_id'].values

word_freq_df = pd.DataFrame()
word_freq_df = word_freq_df.append(df.drop(yes_ids).sum(axis=0).apply(lambda x : log10((x + alpha)/(df.drop(yes_ids).sum(axis=0).sum() + (alpha * len(df.columns))))), ignore_index=True)
word_freq_df = word_freq_df.append(df.drop(no_ids).sum(axis=0).apply(lambda x : log10((x + alpha)/(df.drop(no_ids).sum(axis=0).sum() + (alpha * len(df.columns))))), ignore_index=True)

word_freq_df2 = pd.DataFrame()
word_freq_df2 = word_freq_df2.append(df2.drop(yes_ids).sum(axis=0).apply(lambda x : log10((x + alpha)/(df2.drop(yes_ids).sum(axis=0).sum() + (alpha * len(df2.columns))))), ignore_index=True)
word_freq_df2 = word_freq_df2.append(df2.drop(no_ids).sum(axis=0).apply(lambda x : log10((x + alpha)/(df2.drop(no_ids).sum(axis=0).sum() + (alpha * len(df2.columns))))), ignore_index=True)
    
    

In [176]:
word_freq_df

,00,000,005,07fsra2ny5,0am2j0zhfe,10,100,10k,10mins,11,...,ªgermany,ªsweden,ªð,³ð,¹austria,ºð,¼19,孙春兰,우리의응원을이어주세요,힘내세요_아미
0,-3.569768,-3.569768,-5.574089,-5.574089,-5.574089,-3.569768,-5.574089,-5.574089,-3.569768,-5.574089,...,-5.574089,-5.574089,-5.574089,-5.574089,-5.574089,-5.574089,-5.574089,-5.574089,-3.569768,-3.569768
1,-3.645241,-2.993713,-3.944116,-3.944116,-3.944116,-3.102719,-3.645241,-3.944116,-5.948437,-3.645241,...,-3.944116,-3.944116,-3.944116,-3.944116,-3.944116,-3.944116,-3.944116,-3.944116,-5.948437,-5.948437


In [177]:
word_freq_df2

,00,000,10,100,11,12,14,15,16,17,...,wuhan,year,years,yes,yet,you,young,your,yourself,youâ
0,-3.483724,-3.483724,-3.483724,-5.488046,-5.488046,-3.483724,-5.488046,-5.488046,-3.483724,-5.488046,...,-3.009479,-3.184850,-3.483724,-3.184850,-3.009479,-1.739780,-5.488046,-2.311665,-5.488046,-5.488046
1,-3.560271,-2.908742,-3.017749,-3.560271,-3.560271,-3.859145,-3.384900,-3.260322,-3.260322,-3.560271,...,-3.560271,-3.084592,-2.959834,-3.859145,-3.384900,-2.057219,-3.260322,-2.584485,-3.384900,-3.560271


In [191]:
fv_accuracy = 0
fv_yes_tp = 0;
fv_yes_fp = 0;
fv_yes_tn = 0;
fv_yes_fn = 0;
fv_no_tp = 0;
fv_no_fp = 0;
fv_no_tn = 0;
fv_no_fn = 0;

ov_accuracy = 0
ov_yes_tp = 0;
ov_yes_fp = 0;
ov_yes_tn = 0;
ov_yes_fn = 0;
ov_no_tp = 0;
ov_no_fp = 0;
ov_no_tn = 0;
ov_no_fn = 0;

test_set.columns = training_set.columns
test_cv = CountVectorizer(vocabulary=training_cv.get_feature_names())
sparse = test_cv.fit_transform(test_set['text'])
test_df = pd.DataFrame(sparse.todense(), columns=test_cv.get_feature_names(), index=test_set['tweet_id'])

f = open('trace_NB-BOW-OV.txt', 'w')

for index, values in test_df.iterrows():
    score_yes = log10(len(training_set[training_set['q1_label'] == 'yes']) / len(training_set))
    score_no = log10(len(training_set[training_set['q1_label'] == 'no']) / len(training_set))
    for word in values[values > 0].index:
        score_yes = score_yes + word_freq_df[word][1]
        score_no = score_no + word_freq_df[word][0]
    if score_yes >= score_no:
        c = 'yes'
    else:
        c = 'no'
    real_c = test_set[test_set['tweet_id']== index]['q1_label'].values[0]

    if c == real_c : 
        ov_accuracy += 1
        result = '  correct' 
        if c == 'no':
            ov_no_tp +=1
            ov_yes_tn +=1
        else :
            ov_yes_tp +=1
            ov_no_tn +=1
    else: 
        result = '  wrong'
        if c == 'no':
            ov_no_fp +=1
            ov_yes_fn +=1
        else :
            ov_yes_fp +=1
            ov_no_fn +=1
    f.write(str(index) + '  ' + c + f"  {max(score_yes, score_no):E}  " + real_c + result + '\n')
f.close();

ov_accuracy = ov_accuracy / len(test_set)
ov_yes_precision = ov_yes_tp / (ov_yes_tp + ov_yes_fp)
ov_no_precision = ov_no_tp / (ov_no_tp + ov_no_fp)
ov_yes_recall = ov_yes_tp / (ov_yes_tp + ov_yes_fn)
ov_no_recall = ov_no_tp / (ov_no_tp + ov_no_fn)
ov_yes_f1 = ov_yes_tp / (ov_yes_tp + 0.5 * (ov_yes_fp+ ov_yes_fn) )
ov_no_f1 = ov_no_tp / (ov_no_tp + 0.5 * (ov_no_fp+ ov_no_fn) )

f = open('eval_NB-BOW-OV.txt', 'w')
f.write("%f \n%f  %f\n%f  %f\n%f  %f\n" % (ov_accuracy, ov_yes_precision, ov_no_precision, ov_yes_recall, ov_no_recall, ov_yes_f1, ov_no_f1))
f.close();

test_cv = CountVectorizer(vocabulary=df2.columns.values)
sparse = test_cv.transform(test_set['text'])
test_df2 = pd.DataFrame(sparse.todense(), columns=df2.columns.values, index=test_set['tweet_id'])

f = open('trace_NB-BOW-FV.txt', 'w')


for index, values in test_df2.iterrows():
    score_yes = log10(len(training_set[training_set['q1_label'] == 'yes']) / len(training_set))
    score_no = log10(len(training_set[training_set['q1_label'] == 'no']) / len(training_set))
    for word in values[values > 0].index:
        score_yes += word_freq_df2[word][1]
        score_no += word_freq_df2[word][0]
    if score_yes >= score_no:
        c = 'yes'
    else:
        c = 'no'
        
    real_c = test_set[test_set['tweet_id']== index]['q1_label'].values[0]

    if c == real_c : 
        fv_accuracy += 1
        result = '  correct' 
        if c == 'no':
            fv_no_tp +=1
            fv_yes_tn +=1
        else :
            fv_yes_tp +=1
            fv_no_tn +=1
    else: 
        result = '  wrong'
        if c == 'no':
            fv_no_fp +=1
            fv_yes_fn +=1
        else :
            fv_yes_fp +=1
            fv_no_fn +=1
    f.write(str(index) + '  ' + c + f"  {max(score_yes, score_no):E}  " + real_c + result + '\n')
f.close();

fv_accuracy = fv_accuracy / len(test_set)
fv_yes_precision = fv_yes_tp / (fv_yes_tp + fv_yes_fp)
fv_no_precision = fv_no_tp / (fv_no_tp + fv_no_fp)
fv_yes_recall = fv_yes_tp / (fv_yes_tp + fv_yes_fn)
fv_no_recall = fv_no_tp / (fv_no_tp + fv_no_fn)
fv_yes_f1 = fv_yes_tp / (fv_yes_tp + 0.5 * (fv_yes_fp+ fv_yes_fn) )
fv_no_f1 = fv_no_tp / (fv_no_tp + 0.5 * (fv_no_fp+ fv_no_fn) )

f = open('eval_NB-BOW-FV.txt', 'w')

f.write("%f \n%f  %f\n%f  %f\n%f  %f\n" % (fv_accuracy, fv_yes_precision, fv_no_precision, fv_yes_recall, fv_no_recall, fv_yes_f1, fv_no_f1))
f.close();

In [182]:
from sklearn.naive_bayes import MultinomialNB

In [192]:
clf = MultinomialNB(alpha=.01).fit(df2, training_set['q1_label'])
pred = clf.predict(test_df2)
pred

array(['yes', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'no', 'yes', 'yes',
       'no', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'no', 'yes',
       'yes', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'no',
       'no', 'no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'yes',
       'yes', 'yes', 'yes', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'yes',
       'yes', 'yes', 'no', 'yes'], dtype='<U3')

In [193]:
nb_accuracy = 0;

for i in range(len(pred)):
    if pred[i] == test_set['q1_label'][i]:
        nb_accuracy += 1
        
nb_accuracy / len(pred)

0.7222222222222222